In [1]:
from __future__ import division, print_function

%matplotlib inline
%config InlineBackend.figure_format = "retina"

from matplotlib import rcParams
rcParams["savefig.dpi"] = 100
rcParams["font.size"] = 20

In [2]:
import os
import h5py
import triangle
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
from subprocess import check_call
from scipy.stats import gaussian_kde 

In [31]:
candidates = np.array(open("candidates.txt", "r").readlines(), dtype=int)

In [32]:
candidates

array([201208431, 201257461, 201295312, 201338508, 201367065, 201384232,
       201393098, 201403446, 201445392, 201465501, 201505350, 201546283,
       201549860, 201555883, 201565013, 201569483, 201577035, 201596316,
       201613023, 201617985, 201629650, 201635569, 201649426, 201702477,
       201736247, 201754305, 201779067, 201828749, 201855371, 201912552,
       201929294])

In [102]:
def calc_hist(x, w, fmt):
    q = triangle.quantile(x, [0.16, 0.5, 0.84], weights=w)
    mu = np.sum(x * w) / np.sum(w)
    std = np.sqrt(np.sum((x - mu)**2 * w) / np.sum(w))
    v = "{{0:{0}}} \pm {{{{{{1:{0}}}}}}}".format(fmt).format(mu, std)
#     d = np.diff(q)
#     v = ("{{0:{0}}}_{{{{-{{1:{0}}}}}}}^{{{{+{{2:{0}}}}}}}"
#          .format(fmt).format(q[1], d[0], d[1]))
    return q, v

def plot_hist(x, w, nm, units, fmt=".2f"):
    q, v = calc_hist(x, w, fmt)
    d = np.diff(q)
    bins = np.linspace(q[1] - 3 * d[0], q[1] + 3 * d[1], 12)
    
    fig, ax = pl.subplots()
    ax.hist(x, bins, weights=w, color="k", histtype="step", normed=True)
    ax.axvline(q[1], color="k")
    ax.axvline(q[0], color="k", linestyle="dashed")
    ax.axvline(q[2], color="k", linestyle="dashed")
    if units is None:
        ax.set_xlabel("${0}$".format(nm))
    else:
        ax.set_xlabel("${0}\,[{1}]$".format(nm, units))
    ax.set_yticklabels([])
    if units is not None:
        v = "$" + nm + " = " + v + " \, {0}$".format(units)
    else:
        v = "$" + nm + " = " + v + "$"
    ax.set_title(v)

In [109]:
d = "starmodels_inflated"

def process_row(epic):
    fn = "stlr/{1}/{0}.h5".format(int(epic), d)
    if not os.path.exists(fn):
        try:
            os.makedirs("stlr/" + d)
        except os.error:
            pass
        check_call("wget -O {0} http://www.astro.princeton.edu/~tdm/k2/{2}/{1}.h5"
                   .format(fn, int(epic), d), shell=True)

    # Load the stellar samples and build the Gaussian model.
    star = pd.read_hdf(fn, "samples")
    X = np.array(star[["radius", "mass"]])
    mu = np.mean(X, axis=0)
    cov = np.cov(X, rowvar=0)
    print(mu, np.sqrt(np.diag(cov)))
    
    # KDE shite.
    # kde = gaussian_kde(np.array(star[["radius", "mass"]]).T)
    # Plot the KDE model.
    #     x = np.linspace(star.radius.min(), star.radius.max(), 75)
    #     y = np.linspace(star.mass.min(), star.mass.max(), 77)
    #     X, Y = np.meshgrid(x, y, indexing="ij")
    #     positions = np.vstack([X.ravel(), Y.ravel()]) 
    #     Z = kde(positions).reshape((len(x), len(y)))
    #     pl.pcolor(X, Y, Z, cmap="gray_r")
    #     pl.plot(star.radius, star.mass, ".r", alpha=0.01)

    # Load the samples and convert them to a DataFrame.
    fn = "../ketu/char/{0}/chain.h5".format(int(epic))
    with h5py.File(fn, "r") as f:
        s0 = f["chain"][::5, :, :]
    cols = ["ln_R_star", "ln_M_star", "q_1", "q_2"]
    i = 1
    while len(cols) < s0.shape[-1]:
        cols += [k + "_{0}".format(i) for k in
                 ("ln_R_p", "ln_P", "t_0", "b", "sqrt_e_sin_omega", "sqrt_e_cos_omega")]
        i += 1
    nplanets = i - 1
    dtype = [(k, np.float64) for k in cols]
    samples = pd.DataFrame.from_items(zip(cols, s0.reshape((-1, s0.shape[-1])).T))
    
    # Supplement the samples with Teff measurements drawn from the stellar properties.
    te = np.array(star.Teff)
    inds = np.random.randint(len(te), size=len(samples))
    samples["teff"] = te[inds]
    
    r = (np.exp(np.array(samples[["ln_R_star", "ln_M_star"]])) - mu).T
    lnw = -0.5 * np.sum((np.linalg.solve(cov, r) * r) ** 2, axis=0)
    
    # lnw = np.log(kde(np.exp(np.array(samples[["ln_R_star", "ln_M_star"]])).T))
    
    # Dartmouth model.
    #     # Compute probability under the measured stellar parameters.
    #     lnw = ((np.exp(samples.ln_R_star) - row.Rad) / row.e_Rad) ** 2
    #     lnw += ((np.exp(samples.ln_M_star) - row.M) / row.e_M) ** 2
    #     lnw *= -0.5
    
    # Inculde the interim prior that was 1/R and 1/M.
    lnw += samples.ln_R_star + samples.ln_M_star
    
    # Use this to compute the weights for the samples.
    samples["weights"] = np.exp(lnw + lnw.max())
    
    rows = []
    for i in range(nplanets):
        r = ["{0}".format(int(epic)), "{0}".format(i+1)]
        w = np.array(samples.weights)

        # Plot the period measurement.
        x = np.exp(np.array(samples["ln_P_{0}".format(i+1)]))
        _, v = calc_hist(x, w, ".4f")
        r.append(v)
        # plot_hist(x, w, "P_{{{0}}}".format(i+1), "\mathrm{days}", fmt=".5f")

        # Plot the radius measurement.
        x = np.exp(np.array(samples["ln_R_p_{0}".format(i+1)])) / 0.009155
        _, v = calc_hist(x, w, ".2f")
        r.append(v)
        # plot_hist(x, w, "R_{{p,{0}}}".format(i+1), "R_\oplus", fmt=".2f")
        
        # Plot the a/R_* measurement.
        # Newton's constant in $R_\odot^3 M_\odot^{-1} {days}^{-2}$.
        _G = 2945.4625385377644
        lnp = samples["ln_P_{0}".format(i+1)]
        lnms = samples.ln_M_star
        a = np.array(_G * np.exp(2*lnp+lnms)/(4*np.pi*np.pi)) ** (1./3)
        
        _, v = calc_hist(a * np.exp(-samples.ln_R_star), w, ".2f")
        r.append(v)
        
        _, v = calc_hist(a / 215.1, w, ".4f")
        r.append(v)
        # plot_hist(a / 215.1, w, "{{a}}_{{{0}}}".format(i+1), "\mathrm{AU}", fmt=".4f")
        
        # Teq
        teq = samples.teff / np.sqrt(2. * a * np.exp(-samples.ln_R_star))
        _, v = calc_hist(teq, w, ".0f")
        r.append(v)
        # plot_hist(teq, w, "{{T_\mathrm{{eq}}}}_{{{0}}}".format(i+1), "\mathrm{K}", fmt=".0f")
        
        rows.append(" & ".join(r) + " \\")
    return rows

In [110]:
rows = [process_row(e) for e in candidates]

[ 0.60934138  0.63640491] [ 0.03164453  0.03901752]
[ 0.63464109  0.66993953] [ 0.04760949  0.05276432]
[ 1.02347429  1.045045  ] [ 0.10071951  0.0738505 ]
[ 0.56410861  0.58465262] [ 0.04694068  0.0531474 ]
[ 0.53666077  0.55706176] [ 0.04333041  0.04585901]
[ 0.95124337  0.98319588] [ 0.10157357  0.07395347]
[ 0.91110121  0.96510059] [ 0.0772032   0.06538347]
[ 1.14899452  1.12139758] [ 0.16926021  0.11492552]
[ 0.69898822  0.74594342] [ 0.05264971  0.0608577 ]
[ 0.31529289  0.3193461 ] [ 0.07693887  0.08847471]
[ 0.79198862  0.83070025] [ 0.14043124  0.07833761]
[ 0.78437046  0.84338938] [ 0.06740698  0.06960368]
[ 0.66628104  0.70490952] [ 0.03385958  0.04400451]
[ 0.57840644  0.60281512] [ 0.04464628  0.05005785]
[ 0.51012742  0.53097229] [ 0.07093905  0.07690906]
[ 0.7637729  0.8212061] [ 0.05312153  0.05951604]
[ 0.89218389  0.92929604] [ 0.14094443  0.07124389]
[ 0.75884301  0.82117373] [ 0.07290014  0.08119595]
[ 0.89810132  0.94927357] [ 0.08753707  0.07047404]
[ 0.46758586  

In [111]:
txt = r"""
\clearpage
%\LongTables
\begin{deluxetable*}{lcccccc}
\tablewidth{0pt}
\tabletypesize{\scriptsize}
\tablecaption{Planet Properties for all Objects of Interest \label{Tab:Planets}}
\tablehead{
\colhead{EPIC} &
\colhead{Cand. Num.} &
\colhead{Period (days)} &
\colhead{Radius ($R_\oplus$)} &
\colhead{$a/R_\star$} &
\colhead{$a$ (AU)} &
\colhead{$T_\mathrm{eq}$ (K)}
}
\startdata
DATADATA
\enddata
\tablecomments{blurgh.}
\end{deluxetable*}
""".replace("DATADATA", "\n".join(["$" + r.replace(" & ", "$ & $")[:-1] + r"$ \\" for _ in rows for r in _]))

In [112]:
with open("table_planets.tex", "w") as f:
    f.write(txt)